In [1]:
import cv2
import numpy as np

In [2]:
image = cv2.imread("1.jpeg")
image = cv2.resize(image,(600,600))
image.shape

(600, 600, 3)

In [3]:
height,width,_ = image.shape

In [4]:
cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
## Setup the neural network
net = cv2.dnn.readNet("yolov3.weights",'yolov3.cfg')

In [6]:
classes = []
with open('coco.names','r') as f:
    classes = f.read().splitlines()

In [7]:
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [8]:
### Preprocessing

blob = cv2.dnn.blobFromImage(image,1/255,(416,416),(0,0,0),swapRB=True,crop=False)

In [9]:
for each in blob:
    for n, img in enumerate(each):
        cv2.imshow(str(n),img)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()

In [10]:
net.setInput(blob)
outputLayerNames = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(outputLayerNames)

In [11]:
print(layerOutputs)

[array([[0.04031974, 0.0560901 , 0.47165573, ..., 0.        , 0.        ,
        0.        ],
       [0.0451518 , 0.03388063, 0.36898315, ..., 0.        , 0.        ,
        0.        ],
       [0.04556468, 0.03704954, 0.8661982 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9595505 , 0.95375794, 0.49031338, ..., 0.        , 0.        ,
        0.        ],
       [0.9568873 , 0.96510994, 0.33883888, ..., 0.        , 0.        ,
        0.        ],
       [0.96182245, 0.95452076, 0.82965493, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02008905, 0.02364326, 0.04815596, ..., 0.        , 0.        ,
        0.        ],
       [0.01737675, 0.01791564, 0.3915682 , ..., 0.        , 0.        ,
        0.        ],
       [0.02181257, 0.01803625, 0.07355421, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97174895, 0.9796613 , 0.04613624, ..., 0.        , 0.        ,
        0.        ],
       [0.9826234 

In [12]:
boxes = []
confidences = []
class_ids = []

for each in layerOutputs:
    for detection in each:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
            x = int(center_x - (w/2))
            y = int(center_y - (h/2))
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
print(boxes)
print(confidences)
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.4)
print(indexes.flatten())

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0,255,size = (len(boxes),3))

for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i],2))
    color = colors[i]
    cv2.rectangle(image,(x,y),(x+w,y+h),color,2)
    cv2.putText(image,label+" " + confidence, (x,y+20),font,2,(255,255,255),)
    

[[82, 77, 358, 255], [137, 211, 305, 342]]
[0.9952497482299805, 0.9981611371040344]
[1 0]


In [13]:
cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()